# Hyperparameter Optimization

## Introduction
We previously built and trained a K-Nearest Neighbors classifier using scikit-learn. We also learned about the importance of a validation set.

Now we'll learn how to improve a machine learning model's performance.

We'll use the same dataset as last time. [The Bank Marketing Dataset](../data/subscription_prediction.csv) consists of the following columns:

In [3]:
# import the pandas library
import pandas as pd

# Load the dataset
banking_df = pd.read_csv('../data/subscription_prediction.csv')

# Convert the target column into binary values
banking_df['y'] = banking_df['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Convert the categorical columns into dummy variables
banking_df = pd.get_dummies(data = banking_df, drop_first = True)

                        

## 2. Feature Selection
We previously either used all the features in our dataset to build and train a model or randomly selected a handful of them.

While both can be appropriate approaches to train a model, they aren't necessarily the best ones. We've already observed how our models didn't always perform as well as we'd hoped in either of those scenarios.

Not all features in a dataset might be relevant to a model's performance. Identifying and removing such features in the data preparation step, before training a model, can not only boost its performance, but also reduce the computational cost. The latter is especially important when we have to work with large datasets and complex machine learning models.

There are several ways we can identify and select relevant features:

1. Random selection.
    - We've utilized this in the previous lesson by selecting features at random.
2. Domain Expertise.
    - For example, one of the features is `euribor3m`. A reasonable understanding of what Euribor is could inform of us whether it is likely to have any impact on the prediction.
3. Identifying features that are strongly correlated to our target variable.

We'll learn about other approaches later. For now, we'll calculate the `Pearson Correlation Coefficient` on our columns to identify which features are strongly correlated to the target variable.

We could also plot the heat map for those values to make it easier to identify those features. Since the categorical columns of our dataset have been one-hot encoded, we have over fifty features in our dataset right now. Creating a heat map using all those pairs will make it difficult to identify correlations.

Instead, we'll select five features based on the highest absolute values of Pearson's r.

In [5]:
# Import the train_test_split function for splitting the data set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Calculate the Pearson Correlation Coefficient
correlations = abs(banking_df.corr())

# Identify top 5 features, excluding y itself, that correlate strongly with y.
top_5_features = correlations["y"].sort_values(ascending=False)[1:6].index

# Print the top 5 features
print(correlations["y"].sort_values(ascending=False)[1:6])

# Split the data set into features and target
X = banking_df.drop(["y"], axis=1)
y = banking_df["y"]

# Split the data set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X[top_5_features], y, test_size=0.20, random_state = 417)

# Split the training set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20*X.shape[0]/X_train.shape[0], random_state = 417)

# Scale the features
scaler = MinMaxScaler()

# Scale the training and test sets
X_train_scaled = scaler.fit_transform(X_train)

nr.employed     0.468524
duration        0.468197
euribor3m       0.445328
emp.var.rate    0.429680
pdays           0.317997
Name: y, dtype: float64


## 3. Training and Evaluating a Model

We previously selected some features based on how strongly they correlated to the target variable. It's important to note that calculating the Pearson's r is not an ideal approach.

It's a measure of linear correlation between variables. Therefore, it will fail to capture any non-linear relationships. Additionally, the dummy variables aren't technically continuous. There are alternative approaches (such as Cramér's V) that are more suitable than Pearson's correlation coefficient when working with categorical columns. We won't discuss that here, however.

Let's train our model and then evaluate it on the validation set. We'll try multiple values for 
$K$ when evaluating our model.

In [6]:
# Import the KNeighborsClassifier class
from sklearn.neighbors import KNeighborsClassifier

# Create a list of values for K
num_neighbors = [num for num in range(1, 6)]

# Scale the validation set
X_val_scaled = scaler.transform(X_val)

# Initialize an empty dictionary to store the accuracies
accuracies = {}

# Train the model for each value of K and store the accuracies
for neighbors in num_neighbors:
    knn = KNeighborsClassifier(n_neighbors = neighbors)
    knn.fit(X_train_scaled, y_train)
    val_accuracy = knn.score(X_val_scaled, y_val)
    accuracies[neighbors] = val_accuracy

print(accuracies)

{1: 0.8390123456790124, 2: 0.8093827160493827, 3: 0.8646913580246913, 4: 0.860246913580247, 5: 0.8671604938271605}


## 4. Hyperparameter Optimization

Our validation accuracy changes depending on the number of neighbors ($K$) that we set for our model.

There are certain parameters that we can set or input ourselves when training machine learning models. These parameters can influence the training process and can have an impact on the model's performance. They are called as hyperparameters. For $k$-nearest neighbors, $K$ is one such hyperparameter.

We've tried several values for $K$ so far. In the previous lesson, $k$ = 2000 resulted in a poorly performing model. On the previous screen, however, $k$ = 5 yielded a relatively good accuracy score.

This process of tuning the hyperparameter values in order to maximize the model's performance is called hyperparameter tuning or hyperparameter optimization.

Finding optimal hyperparameter values often requires experimentation. Different models might have a wide range of hyperparameters to tune. The documentation for scikit-learn's KNeighborsClassifier lists multiple parameters that we can set values for when training our model.

We'll experiment with a few of those and observe how they impact our model's performance.


In [11]:
# Import the KNeighborsClassifier class
from sklearn.neighbors import KNeighborsClassifier

# Create a list of values for K
num_neighbors = [num for num in range(1, 6)]

# Scale the validation set
X_val_scaled = scaler.transform(X_val)

# Initialize an empty dictionary to store the accuracies
accuracies = {}

# Train the model for each value of K and store the accuracies
for neighbors in num_neighbors:
    # Train the model for each value of K where power parameter p is set to 5
    knn = KNeighborsClassifier(n_neighbors = neighbors, weights = "distance", p = 5)
    knn.fit(X_train_scaled, y_train)
    val_accuracy = knn.score(X_val_scaled, y_val)
    accuracies[neighbors] = val_accuracy
    
print(accuracies)

{1: 0.8404938271604938, 2: 0.8419753086419753, 3: 0.8523456790123457, 4: 0.8572839506172839, 5: 0.8627160493827161}


## 5. Experimentation vs Fundamentals

Before moving forward, let's look at one of the hyperparameters for our model. On the previous screen, we set the weights to distance.

But what does that do? The documentation presents an explanation, but how do we decide that a particular value makes sense for our use-case?

This will often come through trial and error, but having a better understanding of the algorithm behind our model can give us insights for reflection.

![image](../images/5.1-m739.svg)

We already know we can calculate the distance between an uncategorized data point and all the other data points, identify the $K$ nearest neighbors, and--based on the classes of those neighbors--classify the uncategorized data point by selecting the most common class.

Each neighbor in the above algorithm is given the same weight--no neighbor is more important or relevant than another. When creating our model, this is the same as setting the weights parameter to uniform. All neighbors have a uniform impact when our model decides on a class for the new data point.

Is that what we want every time? What if our dataset had 1000 points corresponding to one label and only 200 corresponding to the other?

We might end up with a higher likelihood of selecting the former class instead of the latter if all neighbors were considered equally. Our model would be biased towards one class. A potential solution to that is to assign weights to those neighbors.

We calculate the distance of each neighbor from the unknown data point. We assign a weight equal to the inverse of that distance. The closer the neighbor to the new point, the more likely the new point belongs to the same class.

This comes with its own potential drawbacks. For example, the additional computations will add to our computational costs. The model might also overfit the data, since it would start to learn from the closest features instead of generalizing appropriately.

If we didn't fully understand how k-NNs work, we might not have understood this specific example of a weighted k-NN, or thought about where it could be potentially useful. Hyperparameter optimization allows us to quickly iterate through different values without thinking about how they impact the model. But it's still important to understand the underlying algorithm as often as possible.

## 6. Grid Search

Previously, modifying two more hyperparameters improved our model's performance corresponding to some of the $K$, but the rest worsened. Not every attempt will result in improvement.

We can't always try every possible permutation and combination. Depending on the size of the dataset, the number of hyperparameters, and the range of values they could take, it would be computationally expensive.

We can try out a smaller subset of values. A commonly used approach that can help us find the optimal hyperparameter values is called grid search.

We've already applied this technique, in part. In the grid search technique, we define a grid of hyperparameter values. This grid contains the range of values for different hyperparameters we want to explore and train our model with.

We created a list containing the number of neighbors we wanted to use to train our model. That was an example of such a grid. We then expanded the grid by setting the values for two more hyperparameters. We could expand the grid further by having multiple nested loops to explore different combinations of hyperparameters and their values.

We don't need to keep creating multiple loops ourselves. We can use Scikit-learn's GridSearchCV to conduct this search.

GridSearchCV allows us to input a dictionary of hyperparameters and the values we want to search. Additionally, GridSearchCV automatically evaluates the different models on validation sets it creates from the training data. It simplifies our workflow in that regard.

Let's utilize this grid search approach next.


In [14]:
# Import the necessary libraries and classes
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Reload the dataset
banking_df = pd.read_csv("../data/subscription_prediction.csv")
banking_df["y"] = banking_df["y"].apply(lambda x: 1 if x=="yes" else 0)
banking_df = pd.get_dummies(data = banking_df, drop_first = True)

# Calculate the Pearson Correlation Coefficient
correlations = abs(banking_df.corr())

# Identify top 5 features, excluding y itself, that correlate strongly with y.
top_5_features = correlations["y"].sort_values(ascending=False)[1:6].index

# Split the data set into features and target
X = banking_df.drop(["y"], axis=1)
y = banking_df["y"]

# Split the data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X[top_5_features], y, test_size=0.20, random_state = 417)

# Scale the training and test sets
scaler = MinMaxScaler()

# Scale the training and test sets
X_train_scaled = scaler.fit_transform(X_train)

# Define the grid of hyperparameters to search
grid_params = {"n_neighbors": range(1, 10),
                "metric": ["minkowski", "manhattan"]
              }

# Initialize the K-Nearest Neighbors classifier
knn = KNeighborsClassifier()

# Initialize the GridSearchCV object
knn_grid = GridSearchCV(knn, grid_params, scoring='accuracy')

# Train the model with the grid of hyperparameters
knn_grid.fit(X_train_scaled, y_train)

# Get the best score and best parameters
best_score = knn_grid.best_score_
best_params = knn_grid.best_params_

# Print the best score and best parameters
print(f"Best model's accuracy: {best_score*100:.2f}")
print(f"Best model's parameters: {best_params}")

Best model's accuracy: 87.70
Best model's parameters: {'metric': 'minkowski', 'n_neighbors': 9}


## 7. Evaluating model on test set
Because of the grid search technique and the features we selected earlier, we were able to obtain a model that has:

1. An accuracy of ~87.7% on the test set.
2. The following hyperparameters and values:
    - metric = "manhattan"
    - n_neighbors: 9
    - Note that your results might vary depending on your experimentation and random seed.

As a reminder: the above model was evaluated on the training data during the hyperparameter grid search and resulted in the highest accuracy compared to all the other models obtained from that hyperparameter grid.

We can now use this model and evaluate it on the test set. Scikit-learn again makes this simple for us to do:

1. We can obtain our best model, known as an estimator, from GridSearchCV.
2. We can evaluate the test set by calculating the accuracy score using the best estimator.

Before we evaluate the model, let's look at how the machine learning workflow has evolved over these lessons:

![image](../images/7.1-m739.svg)

In [19]:
# import necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# reload the dataset
banking_df = pd.read_csv("../data/subscription_prediction.csv")
banking_df["y"] = banking_df["y"].apply(lambda x: 1 if x=="yes" else 0)
banking_df = pd.get_dummies(data = banking_df, drop_first = True)

# calculate the Pearson Correlation Coefficient
correlations = abs(banking_df.corr())

# identify top 5 features that correlate strongly with y
top_5_features = correlations["y"].sort_values(ascending=False)[1:6].index

# split the data set into features and target
X = banking_df.drop(["y"], axis=1)
y = banking_df["y"]

# split the data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X[top_5_features], y, test_size=0.20, random_state = 417)

# scale the training and test sets
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# define the grid of hyperparameters to search
grid_params = {"n_neighbors": range(1, 10),
                "metric": ["minkowski", "manhattan"]
              }

# initialize the K-Nearest Neighbors classifier
knn = KNeighborsClassifier()

# initialize the GridSearchCV object
knn_grid = GridSearchCV(knn, grid_params, scoring='accuracy')
knn_grid.fit(X_train_scaled, y_train)

# scale the test set
X_test_scaled = scaler.transform(X_test)

# evaluate the model on the test set
accuracy = knn_grid.best_estimator_.score(X_test_scaled, y_test)
print(f" Model Accuracy on test set: {accuracy*100:.2f}")

 Model Accuracy on test set: 86.62


With an accuracy of 87%, our final model is much better than the ones we trained in the previous lessons!

In this lesson, we learned that there are two ways we can improve our model's performance:

Identifying and selecting relevant features for training our model.
Tuning or optimizing hyperparameters.
Next, you'll apply everything you've learned in a Guided Project!